In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, SimpleRNN, RNN, LSTM, GRU, SpatialDropout1D, Dropout

In [2]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt

In [3]:
import os, os.path, sys

In [4]:
path = r"C:\Users\mata2\Desktop\master\podaci\0k\X"

In [5]:
dir = os.listdir(path)

Napravljen je pandas DataFrame gde su atributi zasebno pokrenute putanje, dok ce ciljna promenljiva da bude srednja vrednost svih putanja

In [6]:
file_names = []
for file in dir:
    file_names.append(file)

In [7]:
percent_train = 0.67

In [8]:
train_files_names = file_names[:int(percent_train*len(file_names))]
test_file_names = file_names[int(percent_train*len(file_names))::]

Ide do 15 trening skup, posle do 15-20 validacija

In [9]:
kord_sum = [0 for x in range(61000)]

for file in train_files_names[:15]:
    current_file = pd.read_csv(path+'/'+file, header=None)[:61000].astype('int')

    for i in range(len(current_file)):
        kord_sum[i] += current_file[0][i]
average_kord_train = [x//len(train_files_names[:15]) for x in kord_sum]

kord za validacioni skup

In [10]:
kord_sum = [0 for x in range(61000)]

for file in train_files_names[15:20]:
    current_file = pd.read_csv(path+'/'+file, header=None)[:61000].astype('int')

    for i in range(len(current_file)):
        kord_sum[i] += current_file[0][i]
average_kord_test = [x//len(train_files_names[15:20]) for x in kord_sum]

In [11]:
train__ = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in train_files_names], axis=0)

Test skup je od 15 fajlova, dok je validacioni od 5
Validacioni skup = test_

In [12]:
test_ = train__[61000*15::]

In [13]:
train_ = train__[:61000*15]

In [14]:
for j in range(int(len(train_)/61000)):
    exec(f"razlika_trening_{j} = []")
    for i in range(61000):
        exec(f"razlika_trening_{j}.append(average_kord_train[i] - train_[j*61000+i])")

In [15]:
for j in range(int(len(test_)/61000)):
    exec(f"razlika_test_{j} = []")
    for i in range(61000):
        exec(f"razlika_test_{j}.append(average_kord_test[i] - train_[j*61000+i])")

In [20]:
train_podaci = []
for j in range(int(len(train_)/61000)):
    for i in range(61000):
        exec(f'train_podaci.append(razlika_trening_{j}[i][0])')

In [21]:
test_podaci = []
for j in range(int(len(test_)/61000)):
    for i in range(61000):
        exec(f'test_podaci.append(razlika_test_{j}[i][0])')

In [17]:
def create_timeseries_split(data, time_steps):

  # Get the number of samples in the data
  num_samples = len(data) - time_steps

  # Create empty arrays to store features and target values
  x_train = np.zeros((num_samples, time_steps, 1))
  y_train = np.zeros((num_samples, 1))

  # Loop through the data and create features and target values
  for i in range(num_samples):
    # Extract a slice of data for the current feature
    x_train[i] = data[i:i+time_steps, :]

    # The target value is the next value after the feature
    y_train[i] = data[i+time_steps, 0]

  return x_train, y_train

In [22]:
train_podaci = np.asarray(train_podaci).reshape(-1,1)
test_podaci = np.asarray(test_podaci).reshape(-1,1) 

In [61]:
time_steps = 5

In [62]:
train_X, train_y = create_timeseries_split(train_podaci, time_steps)
test_X, test_y = create_timeseries_split(test_podaci, time_steps)

In [93]:
input_shape = (time_steps,1)

In [94]:
keras.utils.set_random_seed(7)
optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)
modelMSE = Sequential()
modelMSE.add(GRU(units = 16, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 8, activation=keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 16, activation=keras.layers.LeakyReLU()))
#modelMSE.add(Dense(units = 8, activation=keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)


In [95]:
historyMSE = modelMSE.fit(train_X, train_y, epochs=10, batch_size = 64, verbose = 2)

Epoch 1/10
14297/14297 - 35s - loss: 2.0493 - 35s/epoch - 2ms/step
Epoch 2/10
14297/14297 - 33s - loss: 0.9427 - 33s/epoch - 2ms/step
Epoch 3/10
14297/14297 - 33s - loss: 0.9152 - 33s/epoch - 2ms/step
Epoch 4/10
14297/14297 - 33s - loss: 0.9050 - 33s/epoch - 2ms/step
Epoch 5/10


KeyboardInterrupt: 

In [ ]:
train_scores = modelMSE.evaluate(test_X, test_y)

In [ ]:
train_predict_MSE = modelMSE.predict(train_X)
test_predict_MSE = modelMSE.predict(test_X)

In [29]:
time_steps_list = [1,2,4,5,8,10]

In [31]:
best_step = None
best_skor = float('inf')

for step in time_steps_list:
    train_X, train_y = create_timeseries_split(train_podaci, step)
    test_X, test_y = create_timeseries_split(test_podaci, step)
    input_shape = (step,1)
    
    keras.utils.set_random_seed(7)
    optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)
    modelMSE = Sequential()
    modelMSE.add(GRU(units = 16, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
    modelMSE.add(Dense(units = 8, activation=keras.layers.LeakyReLU()))
    modelMSE.add(Dense(units = 16, activation=keras.layers.LeakyReLU()))
    #modelMSE.add(Dense(units = 8, activation=keras.layers.LeakyReLU()))
    modelMSE.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
    modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)
    
    historyMSE = modelMSE.fit(train_X, train_y, epochs=10, batch_size = 64, verbose = 2)
    train_scores = modelMSE.evaluate(test_X, test_y)
    
    if train_scores < best_skor:
        best_skor = train_scores
        best_step = step
        
        print('best_skor', best_skor, 'best_step',best_step)

print('best_skor', best_skor)
print('best_step', best_step)



Epoch 1/10

14297/14297 - 22s - loss: 2.5902 - 22s/epoch - 2ms/step
Epoch 2/10
14297/14297 - 19s - loss: 1.7841 - 19s/epoch - 1ms/step
Epoch 3/10
14297/14297 - 19s - loss: 1.7834 - 19s/epoch - 1ms/step
Epoch 4/10
14297/14297 - 19s - loss: 1.7834 - 19s/epoch - 1ms/step
Epoch 5/10
14297/14297 - 19s - loss: 1.7834 - 19s/epoch - 1ms/step
Epoch 6/10
14297/14297 - 19s - loss: 1.7831 - 19s/epoch - 1ms/step
Epoch 7/10
14297/14297 - 19s - loss: 1.7829 - 19s/epoch - 1ms/step
Epoch 8/10
14297/14297 - 19s - loss: 1.7830 - 19s/epoch - 1ms/step
Epoch 9/10
14297/14297 - 19s - loss: 1.7827 - 19s/epoch - 1ms/step
Epoch 10/10
14297/14297 - 19s - loss: 1.7828 - 19s/epoch - 1ms/step
9532/9532 [==============================] - 10s 1ms/step - loss: 2.4786
best_skor 2.478580951690674 best_step 1
Epoch 1/10
14297/14297 - 24s - loss: 2.3381 - 24s/epoch - 2ms/step
Epoch 2/10
14297/14297 - 22s - loss: 1.3450 - 22s/epoch - 2ms/step
Epoch 3/10
14297/14297 - 22s - loss: 1.3147 - 22s/epoch - 2ms/step
Epoch 4/10
14

In [32]:
time_steps = 8
input_shape = (time_steps,1)

In [33]:
train_X, train_y = create_timeseries_split(train_podaci, time_steps)
test_X, test_y = create_timeseries_split(test_podaci, time_steps)

# TESTIRANJE ARHITEKTURE i LR/BATCH

In [34]:
gru_unit = [8,16]
unit1 = [8,16]
unit2 = [8,16]
LR = [0.01,0.001]
batch_size = [32,64]

In [36]:
best_skor = float('inf')
best_gru = None
best_unit1 = None
best_unit2 = None
best_batch = None
best_LR = None

for gru in gru_unit:
    for uni1 in unit1:
        for uni2 in unit2:
            #for rate in LR:
            for batch in batch_size:

                keras.utils.set_random_seed(7)
                optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

                model = Sequential()
                model.add(GRU(units = gru, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
                model.add(Dense(units = uni1, activation=keras.layers.LeakyReLU()))
                model.add(Dense(units = uni2, activation=keras.layers.LeakyReLU()))
                model.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
                model.compile(loss='mean_squared_error', optimizer = optimizerMSE)

                model.fit(train_X, train_y, epochs = 10, batch_size = batch, verbose = 2)
                train_scores = model.evaluate(test_X, test_y)

                if train_scores < best_skor:
                    best_skor = train_scores
                    best_gru = gru
                    best_unit1 = uni1
                    best_unit2 = uni2
#                    best_LR = rate
                    best_batch = batch

                    print('best_skor: ', best_skor,'best_gru: ', best_gru,'best_unit1: ', best_unit1,'best_unit2: ', best_unit2,'best batch size: ', best_batch)
print('best_skor: ', best_skor)                    
print('best_gru: ', best_gru)
print('best_unit1: ', best_unit1)
print('best_unit2: ', best_unit2)
#print('best learning rate: ', best_LR)    
print('best batch size: ', best_batch)    


Epoch 1/10
28594/28594 - 85s - loss: 1.8519 - 85s/epoch - 3ms/step
Epoch 2/10
28594/28594 - 81s - loss: 0.9701 - 81s/epoch - 3ms/step
Epoch 3/10
28594/28594 - 82s - loss: 0.9329 - 82s/epoch - 3ms/step
Epoch 4/10
28594/28594 - 84s - loss: 0.9182 - 84s/epoch - 3ms/step
Epoch 5/10
28594/28594 - 94s - loss: 0.9075 - 94s/epoch - 3ms/step
Epoch 6/10
28594/28594 - 94s - loss: 0.8990 - 94s/epoch - 3ms/step
Epoch 7/10
28594/28594 - 95s - loss: 0.8920 - 95s/epoch - 3ms/step
Epoch 8/10
28594/28594 - 92s - loss: 0.8857 - 92s/epoch - 3ms/step
Epoch 9/10
28594/28594 - 90s - loss: 0.8807 - 90s/epoch - 3ms/step
Epoch 10/10
28594/28594 - 93s - loss: 0.8761 - 93s/epoch - 3ms/step
9531/9531 [==============================] - 16s 2ms/step - loss: 1.2181
best_skor:  1.218113899230957 best_gru:  8 best_unit1:  8 best_unit2:  8 best batch size:  32
Epoch 1/10
14297/14297 - 47s - loss: 2.3009 - 47s/epoch - 3ms/step
Epoch 2/10
14297/14297 - 44s - loss: 1.0031 - 44s/epoch - 3ms/step
Epoch 3/10
14297/14297 - 43s

28594/28594 - 72s - loss: 0.8605 - 72s/epoch - 3ms/step
Epoch 6/10
28594/28594 - 72s - loss: 0.8533 - 72s/epoch - 3ms/step
Epoch 7/10
28594/28594 - 72s - loss: 0.8470 - 72s/epoch - 3ms/step
Epoch 8/10
28594/28594 - 72s - loss: 0.8423 - 72s/epoch - 3ms/step
Epoch 9/10
28594/28594 - 72s - loss: 0.8389 - 72s/epoch - 3ms/step
Epoch 10/10
28594/28594 - 72s - loss: 0.8355 - 72s/epoch - 3ms/step
9531/9531 [==============================] - 13s 1ms/step - loss: 1.1513
best_skor:  1.1513310670852661 best_gru:  16 best_unit1:  8 best_unit2:  16 best batch size:  32
Epoch 1/10
14297/14297 - 38s - loss: 2.0699 - 38s/epoch - 3ms/step
Epoch 2/10
14297/14297 - 36s - loss: 0.9437 - 36s/epoch - 3ms/step
Epoch 3/10
14297/14297 - 36s - loss: 0.8987 - 36s/epoch - 3ms/step
Epoch 4/10
14297/14297 - 36s - loss: 0.8772 - 36s/epoch - 3ms/step
Epoch 5/10
14297/14297 - 36s - loss: 0.8635 - 36s/epoch - 3ms/step
Epoch 6/10
14297/14297 - 36s - loss: 0.8543 - 36s/epoch - 3ms/step
Epoch 7/10
14297/14297 - 36s - loss:

In [39]:
batch_size = [64,128,256,512,1024,2048,4096]

In [67]:
keras.utils.set_random_seed(7)
optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)
modelMSE = Sequential()
modelMSE.add(GRU(units = 16, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 16, activation=keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 16, activation=keras.layers.LeakyReLU()))
#modelMSE.add(Dense(units = 8, activation=keras.layers.LeakyReLU()))
modelMSE.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
modelMSE.compile(loss='mean_squared_error', optimizer = optimizerMSE)

In [40]:
historyMAE = modelMAE.fit(train_X, train_y, epochs=10, batch_size = 16, verbose = 2)

NameError: name 'modelMAE' is not defined

In [68]:
historyMSE = modelMSE.fit(train_X, train_y, epochs=10, batch_size = 1024, verbose = 2)

Epoch 1/10
894/894 - 9s - loss: 7173579776.0000 - 9s/epoch - 10ms/step
Epoch 2/10
894/894 - 2s - loss: 48031884.0000 - 2s/epoch - 3ms/step
Epoch 3/10
894/894 - 2s - loss: 1464.7377 - 2s/epoch - 2ms/step
Epoch 4/10
894/894 - 2s - loss: 1464.3621 - 2s/epoch - 2ms/step
Epoch 5/10
894/894 - 2s - loss: 1464.1544 - 2s/epoch - 2ms/step
Epoch 6/10
894/894 - 2s - loss: 1464.0243 - 2s/epoch - 2ms/step
Epoch 7/10
894/894 - 2s - loss: 1463.9316 - 2s/epoch - 3ms/step
Epoch 8/10
894/894 - 2s - loss: 1463.8832 - 2s/epoch - 3ms/step
Epoch 9/10
894/894 - 2s - loss: 1463.6785 - 2s/epoch - 2ms/step
Epoch 10/10
894/894 - 2s - loss: 1463.3251 - 2s/epoch - 2ms/step


In [69]:
#train_scores_MAE = modelMAE.evaluate(test_X, test_y)
train_scores_MSE = modelMSE.evaluate(test_X,test_y)

28594/28594 [==============================] - 37s 1ms/step - loss: 1463.4017


In [70]:
test_predict_MSE = modelMSE.predict(test_X)
#test_predict_MAE = modelMAE.predict(test_X)

28594/28594 [==============================] - 36s 1ms/step


# TESTIRANJE SA VISE INSTANCI